In [6]:
import os
import sys
from antlr4 import *
from ExprLexer import ExprLexer
from ExprParser import ExprParser
from ExprListener import ExprListener
from ExprVisitor import ExprVisitor

print("Import Successful!")

Import Successful!


# listener

In [7]:
class MyExprListener(ExprListener):
    def enterProg(self, ctx:ExprParser.ProgContext):
        print("Entering program")

    def exitProg(self, ctx:ExprParser.ProgContext):
        print("Exiting program")

    def enterStat(self, ctx:ExprParser.StatContext):
        print(f"Entering statement: {ctx.getText()}")

    def exitStat(self, ctx:ExprParser.StatContext):
        print(f"Exiting statement: {ctx.getText()}")

    def enterExpr(self, ctx:ExprParser.ExprContext):
        print(f"Entering expression: {ctx.getText()}")

    def exitExpr(self, ctx:ExprParser.ExprContext):
        print(f"Exiting expression: {ctx.getText()}")


In [11]:
input_path = "examples/expr.txt"

input_stream = FileStream(input_path)
lexer = ExprLexer(input_stream)
stream = CommonTokenStream(lexer)
parser = ExprParser(stream)
tree = parser.prog()

listener = MyExprListener()
walker = ParseTreeWalker()
walker.walk(listener, tree)

ANTLR runtime and generated code versions disagree: 4.10!=4.13.1
ANTLR runtime and generated code versions disagree: 4.10!=4.13.1
Entering program
Entering statement: 193

Entering expression: 193
Exiting expression: 193
Exiting statement: 193

Entering statement: a=5

Entering expression: 5
Exiting expression: 5
Exiting statement: a=5

Entering statement: b=6

Entering expression: 6
Exiting expression: 6
Exiting statement: b=6

Entering statement: a+b*2

Entering expression: a+b*2
Entering expression: a
Exiting expression: a
Entering expression: b*2
Entering expression: b
Exiting expression: b
Entering expression: 2
Exiting expression: 2
Exiting expression: b*2
Exiting expression: a+b*2
Exiting statement: a+b*2

Entering statement: (1+2)*3<missing NEWLINE>
Entering expression: (1+2)*3
Entering expression: (1+2)
Entering expression: 1+2
Entering expression: 1
Exiting expression: 1
Entering expression: 2
Exiting expression: 2
Exiting expression: 1+2
Exiting expression: (1+2)
Entering ex

line 5:7 missing NEWLINE at '<EOF>'


# visitor

In [27]:
class MyExprVisitor(ExprVisitor):
    def visitProg(self, ctx:ExprParser.ProgContext):
        print("Visiting program")
        return self.visitChildren(ctx)

    def visitStat(self, ctx:ExprParser.StatContext):
        print(f"Visiting statement: {ctx.getText()}")
        return self.visitChildren(ctx)

    def visitExpr(self, ctx:ExprParser.ExprContext):
        if ctx.getChildCount() == 1:  # INT or ID
            return ctx.getText()
        elif ctx.getChildCount() == 3:
            left = self.visit(ctx.getChild(0))
            op = ctx.getChild(1).getText()
            right = self.visit(ctx.getChild(2))
            return f"({left} {op} {right})"
        elif ctx.getChildCount() == 2:  # Parens (expr)
            return self.visit(ctx.getChild(1))

    def visitTerminal(self, node):
        return node.getText()

In [28]:
visitor = MyExprVisitor()
result = visitor.visit(tree)
print(f"Result: {result}")

Visiting program
Visiting statement: 193

Visiting statement: a=5

Visiting statement: b=6

Visiting statement: a+b*2

Visiting statement: (1+2)*3<missing NEWLINE>
Result: None


In [31]:
class MyExprVisitor(ExprVisitor):
    def visitProg(self, ctx:ExprParser.ProgContext):
        """
        Visits each statement in the program, collects the results, and returns them as a list.
        """
        results = []
        for stat in ctx.stat():
            result = self.visit(stat)
            if result is not None:
                results.append(result)
        return results

    def visitStat(self, ctx:ExprParser.StatContext):
        """
        Visits a statement. If it's an expression, it visits the expression.
        If it's an assignment (ID = expr), it visits the expression and returns the assignment as a string.
        """
        if ctx.expr():
            return self.visit(ctx.expr())
        elif ctx.ID():
            var_name = ctx.ID().getText()
            value = self.visit(ctx.expr())
            return f"{var_name} = {value}"
        else:
            return None

    def visitExpr(self, ctx:ExprParser.ExprContext):
        """
        Visits expressions. Handles single child nodes (integers or identifiers),
        binary operations (recursively visits left and right operands and returns them with the operator in between),
        and parenthesized expressions (recursively visits the inner expression).
        """
        if ctx.getChildCount() == 1:  # INT or ID
            return ctx.getText()
        elif ctx.getChildCount() == 3:
            left = self.visit(ctx.getChild(0))
            op = ctx.getChild(1).getText()
            right = self.visit(ctx.getChild(2))
            return f"({left} {op} {right})"
        elif ctx.getChildCount() == 2:  # Parens (expr)
            return self.visit(ctx.getChild(1))


In [32]:
visitor = MyExprVisitor()
result = visitor.visit(tree)
print(f"Result: {result}")

Result: ['193', '5', '6', '(a + (b * 2))', '((None 1+2 None) * 3)']


Here's a comparative table highlighting the differences between the Visitor and Listener patterns in ANTLR:

| Aspect                      | Visitor                                    | Listener                                   |
|-----------------------------|--------------------------------------------|--------------------------------------------|
| **Traversal Type**          | Explicit traversal                         | Implicit traversal                         |
| **Entry/Exit Handling**     | You visit nodes explicitly and control traversal. | Entry and exit methods are automatically called for each node. |
| **Method Invocation**       | `visit` method is used to traverse nodes.  | `enter` and `exit` methods are called as the tree is walked. |
| **Node Access**             | You have to explicitly call `visit` on each node you want to process. | Automatically handles traversal based on the grammar rules. |
| **Return Values**           | Can return values from `visit` methods.    | Typically used for side-effects; not designed to return values. |
| **Use Case**                | When you need to transform or compute values from the parse tree. | When you want to perform actions or side-effects based on the parse tree structure. |
| **Code Complexity**         | Can be more complex due to explicit node visits and managing state. | Typically simpler, as you handle nodes as they are visited or exited. |
| **Flexibility**             | High flexibility; you control traversal and can visit nodes in any order. | Less flexible; traversal is handled by the framework and is usually in a depth-first manner. |
| **State Management**        | Explicit management of state and results.  | Typically, state is managed using instance variables or external structures. |
| **Example Use Case**        | Generating a formatted output or evaluating expressions. | Logging or collecting statistics as the tree is traversed. |
| **Grammar Definition**      | Better for complex transformations or evaluations. | Better for simple actions that don't need to manipulate the tree. |

### Visitor Pattern

**When to Use:**
- When you need to perform complex operations on the parse tree.
- When you need to collect results or compute values as you traverse the tree.
- When you want full control over traversal and node processing.

**Advantages:**
- Allows for more complex processing and transformations.
- You can return values and perform computations during traversal.
- More control over traversal order and processing logic.

**Disadvantages:**
- Can be more complex to implement.
- Requires explicit calls to `visit` methods for each node.

### Listener Pattern

**When to Use:**
- When you need to perform actions or side-effects based on node visits.
- When you want to keep the implementation simple and handle nodes as they are visited or exited.
- When you want automatic traversal without managing it explicitly.

**Advantages:**
- Simpler to implement for straightforward actions.
- Automatic handling of traversal based on grammar rules.
- Suitable for tasks like logging or collecting statistics.

**Disadvantages:**
- Less control over traversal and node processing.
- Generally not used for complex transformations or computations.

### Example Comparison

**Visitor Example:**
```python
def visitExpr(self, ctx:ExprParser.ExprContext):
    if ctx.getChildCount() == 1:
        return ctx.getText()
    elif ctx.getChildCount() == 3:
        left = self.visit(ctx.getChild(0))
        op = ctx.getChild(1).getText()
        right = self.visit(ctx.getChild(2))
        return f"({left} {op} {right})"
    elif ctx.getChildCount() == 2:
        return self.visit(ctx.getChild(1))
```

**Listener Example:**
```python
def enterExpr(self, ctx:ExprParser.ExprContext):
    print(f"Entering expression: {ctx.getText()}")

def exitExpr(self, ctx:ExprParser.ExprContext):
    print(f"Exiting expression: {ctx.getText()}")
```

In summary, the Visitor pattern provides more control and flexibility, especially useful for complex processing and transformations. The Listener pattern is more straightforward and well-suited for tasks that involve simple actions or side-effects during tree traversal.

In ANTLR, both `visitor.visit(tree)` and `tree.accept(visitor)` are used to traverse the parse tree, but they differ in their mechanics and typical use cases. Here's a detailed comparison of the two approaches:

### `visitor.visit(tree)`

- **Description**: This method is part of the Visitor pattern. You call `visit` on the visitor instance, passing the root of the parse tree (`tree`). The visitor then recursively visits nodes based on the `visit` methods defined in your visitor class.

- **How It Works**: When you call `visitor.visit(tree)`, the `visit` method on the visitor is invoked with the root of the parse tree. The `visit` method then dispatches the node to the appropriate `visit` method for its specific type. This allows you to perform custom processing or transformations on each node of the tree.

- **Example Usage**:
    ```python
    visitor = ExprVisitorImpl()
    result = visitor.visit(tree)
    ```

- **Typical Use Case**: Use `visitor.visit(tree)` when you want to explicitly control the traversal of the parse tree and perform custom actions or computations at each node.

- **Flexibility**: Provides full control over traversal, allowing you to manage state and transformations in a flexible manner.

### `tree.accept(visitor)`

- **Description**: This method is part of the Visitor pattern as well but is invoked on the parse tree node itself. It delegates the traversal to the visitor’s `visit` method for the node type.

- **How It Works**: When you call `tree.accept(visitor)`, the `accept` method of the parse tree node is invoked. This method typically calls the `visit` method of the visitor instance, passing the current node (`tree`) as an argument. The `accept` method is implemented in the generated parse tree classes (like `ExprContext` in ANTLR).

- **Example Usage**:
    ```python
    result = tree.accept(visitor)
    ```

- **Typical Use Case**: Use `tree.accept(visitor)` when you want the node itself to initiate the traversal, typically when dealing with a specific node rather than starting from the root. It allows nodes to handle their own traversal.

- **Flexibility**: It is convenient for handling nodes individually, but you might need to ensure that each node correctly implements `accept`.

### Detailed Comparison

| Aspect                      | `visitor.visit(tree)`                          | `tree.accept(visitor)`                     |
|-----------------------------|------------------------------------------------|--------------------------------------------|
| **Control Over Traversal**  | Explicitly starts traversal from the root node. | The node itself initiates traversal.       |
| **Invocation**              | Calls `visit` method on the visitor.           | Calls `accept` method on the node, which then calls `visit`. |
| **Typical Use Case**        | General tree traversal, especially when starting from the root. | Traversing a specific node or subtree.     |
| **Flexibility**             | Full control over traversal and state management. | The node handles its traversal; less control if only using `accept`. |
| **Node Handling**           | Uses the visitor's methods to handle each node type. | Delegates to visitor’s methods via the node’s `accept` method. |
| **Implementation Details**  | You manage the traversal explicitly.           | Requires `accept` method to be properly implemented in node classes. |

### Example in Context

**Visitor Pattern with `visitor.visit(tree)`**:
```python
class MyVisitor(ExprVisitor):
    def visitProg(self, ctx):
        for stat in ctx.stat():
            self.visit(stat)

# Create a visitor and traverse the tree from the root
visitor = MyVisitor()
visitor.visit(tree)
```

**Visitor Pattern with `tree.accept(visitor)`**:
```python
class ExprContext:
    def accept(self, visitor):
        return visitor.visitExpr(self)

# Traverse a specific node
result = tree.accept(visitor)
```

In summary, `visitor.visit(tree)` is used to start the traversal of the parse tree from the root and provides explicit control over the traversal. `tree.accept(visitor)` allows individual nodes to handle their own traversal by delegating to the visitor's `visit` methods, which can be useful when dealing with specific nodes or subtrees.